1.	Level  0
2.	Count of  0(False)  =  1
3.	Count of  1(True)  =  3
4.	Current Entropy  is =  0.811278124459
5.	Splitting on feature  X1  with gain ratio  0.311278124459

*************************************************************

1.	Level  1
2.	Count of  0  =  1
3.	Count of  1  =  1
4.	Current Entropy is =  1.0
5.	Splitting on feature  X2  with gain ratio  1.0

*************************************************************

1.	Level  2
2.	Count of  0  =  1
3.	Current Entropy  is =  0.0
4.	Reached leaf Node

*************************************************************

1.	Level  2
2.	Count of  1  =  1
3.	Current Entropy  is =  0.0
4.	Reached leaf Node

*************************************************************

1.	Level  1
2.	Count of  1  =  2
3.	Current Entropy  is =  0.0
4.	Reached leaf Node



In [4]:
import numpy as np

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [6]:
def level(X):
    mean = X.mean()
    one_third_value = mean/2
    two_third_value = 1.5 * mean
    for i in range(len(X)):
        if X[i] < one_third_value:
            X[i] = 1
        elif X[i] < mean:
            X[i] = 2
        elif X[i] < two_third_value:
            X[i] = 3
        else:
            X[i] = 4
    return X

In [47]:
dataset = datasets.load_iris()
X = dataset.data
Y = dataset.target
#for i in range(X.shape[1]):
#    X[:, i] = level(X[:, i])
feature_names = dataset.feature_names
class_names = dataset.target_names
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)


In [42]:
def entropy(Y, possible_classes):
    entropy = 0
    total_count = len(Y)
    for y in possible_classes :
        count_y = (Y == y).sum()
        p_i = count_y/total_count
        entropy -=  p_i * np.log(p_i)
    return entropy

def info_gain_for_cont_data_split(Y, splits):
    info_gain = 0
    split_info = 0    
    mod_Y = len(Y)
    unique_splits = list(set(splits))
    for s in unique_splits:
        indices = (splits == s)
        Y_split_i = Y[indices]
        #print([X_split_i, Y_split_i])
        mod_Y_i = len(Y_split_i)
        ratio_mod_Y_i_to_mod_Y = (mod_Y_i/mod_Y)
        possible_classes_Y_i = list(set(Y_split_i))
        info_gain += ((ratio_mod_Y_i_to_mod_Y) * entropy( Y_split_i, possible_classes_Y_i))
        split_info -= (ratio_mod_Y_i_to_mod_Y * np.log(ratio_mod_Y_i_to_mod_Y))
    return info_gain, split_info

def split(X, Y, feature):
    possible_values_current_feature = list(set( X[:, feature] ))
    pvcf = possible_values_current_feature
    mid_values = []
    
    for i in range(len(possible_values_current_feature) -1 ):
        mid_values.append((pvcf[i] + pvcf[i+1])/2)
    min_info_gain = np.inf
    mid_value_for_min_info_gain = 0
    min_split_info = 0
    best_split_labels = []
    for value in mid_values:
        current_split_labels = np.zeros(len(Y))
        current_split_labels[ X[:, feature] <= value ] = 1
        current_split_information_gain, current_split_split_info = info_gain_for_cont_data_split(Y, current_split_labels)
        #print(current_split_information_gain)
        if current_split_information_gain < min_info_gain :
            min_info_gain = current_split_information_gain
            mid_value_for_min_info_gain = value
            min_split_info = current_split_split_info
            best_split_labels = current_split_labels
    #print("min_info_gain, min_split_info, best_split_labels, mid_value_for_min_info_gain")
    #print(min_info_gain, min_split_info, best_split_labels, mid_value_for_min_info_gain)
    
    return min_info_gain, min_split_info, best_split_labels, mid_value_for_min_info_gain

    
def decision_tree(X, Y, features, level):
    print("\nLevel", level)
    possible_classes = list(set(Y))
    max_count = -np.inf
    max_class = possible_classes[0]
    for y in possible_classes:
        count_y = (Y == y).sum()
        print("Count of", y, "=", count_y)
        if count_y > max_count :
            max_count = count_y
            max_class = y   
    current_entropy = entropy(Y, possible_classes)
    print("Current Entropy is =", current_entropy)
    
    if len(possible_classes) == 1 or len(features) == 0:
        print("Reached leaf Node")
        return max_class
    selected_feature = None
    max_info_gain = -np.inf
    best_split_labels = None
    best_split_mid_value = None
    for f in features:
        info_gain, split_info, current_split, current_mid_value_for_f = split(X, Y, f)
        #print("info_gain, split_info", info_gain, split_info)
        #print("gain ratio", info_gain/split_info)

        if info_gain > max_info_gain:
            selected_feature = f
            max_info_gain = info_gain
            best_split_labels = current_split
            best_split_mid_value = current_mid_value_for_f
    print("Splitting on feature", selected_feature,"with gain ratio", max_info_gain)
    #splits = split(X, Y, selected_feature)
    unique_splits = list(set(best_split_labels))
    
    selected_feature_index = np.argwhere(features == selected_feature)
    features =  np.delete(features, selected_feature_index)
    
    for s in unique_splits:
        indices = (best_split_labels == s)
        X_split_i = X[indices, :]
        Y_split_i = Y[indices]
        decision_tree(X_split_i, Y_split_i, features, level + 1)

#print(split(X, Y, 0))
      
    
features = np.arange(len(feature_names))
max_class = decision_tree(X, Y, features, 0)



Level 0
Count of 0 = 212
Count of 1 = 357
Current Entropy is = 0.660316349195
Splitting on feature 11 with gain ratio 0.651003787895

Level 1
Count of 0 = 209
Count of 1 = 332
Current Entropy is = 0.667074235702
Splitting on feature 18 with gain ratio 0.656164154259

Level 2
Count of 0 = 9
Count of 1 = 1
Current Entropy is = 0.325082973391
Splitting on feature 9 with gain ratio 0.27033672532

Level 3
Count of 0 = 5
Count of 1 = 1
Current Entropy is = 0.450561208866
Splitting on feature 19 with gain ratio 0.374890096413

Level 4
Count of 0 = 2
Current Entropy is = 0.0
Reached leaf Node

Level 4
Count of 0 = 3
Count of 1 = 1
Current Entropy is = 0.562335144619
Splitting on feature 1 with gain ratio 0.34657359028

Level 5
Count of 0 = 2
Current Entropy is = 0.0
Reached leaf Node

Level 5
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 0.69314718056
Splitting on feature 0 with gain ratio 0.0

Level 6
Count of 0 = 1
Current Entropy is = 0.0
Reached leaf Node

Level 6
Count of 1 = 1
Curr


Level 15
Count of 0 = 12
Count of 1 = 7
Current Entropy is = 0.658109987543
Splitting on feature 5 with gain ratio 0.57061783478

Level 16
Count of 0 = 10
Count of 1 = 3
Current Entropy is = 0.540204142389
Splitting on feature 24 with gain ratio 0.490709201931

Level 17
Count of 0 = 7
Count of 1 = 1
Current Entropy is = 0.376770161256
Splitting on feature 3 with gain ratio 0.17328679514

Level 18
Count of 0 = 6
Current Entropy is = 0.0
Reached leaf Node

Level 18
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 0.69314718056
Splitting on feature 0 with gain ratio 0.0

Level 19
Count of 0 = 1
Current Entropy is = 0.0
Reached leaf Node

Level 19
Count of 1 = 1
Current Entropy is = 0.0
Reached leaf Node

Level 17
Count of 0 = 3
Count of 1 = 2
Current Entropy is = 0.673011667009
Splitting on feature 7 with gain ratio 0.381908500977

Level 18
Count of 0 = 2
Current Entropy is = 0.0
Reached leaf Node

Level 18
Count of 0 = 1
Count of 1 = 2
Current Entropy is = 0.636514168295
Splitting on 

Level 7
Count of 0 = 25
Count of 1 = 52
Current Entropy is = 0.630343486945
Splitting on feature 15 with gain ratio 0.562442840999

Level 8
Count of 0 = 25
Count of 1 = 40
Current Entropy is = 0.666278442415
Splitting on feature 4 with gain ratio 0.553925882499

Level 9
Count of 0 = 25
Count of 1 = 27
Current Entropy is = 0.692407353123
Splitting on feature 24 with gain ratio 0.582351131654

Level 10
Count of 0 = 17
Count of 1 = 6
Current Entropy is = 0.573964491393
Splitting on feature 29 with gain ratio 0.479879218818

Level 11
Count of 0 = 6
Current Entropy is = 0.0
Reached leaf Node

Level 11
Count of 0 = 11
Count of 1 = 6
Current Entropy is = 0.649248354871
Splitting on feature 1 with gain ratio 0.511689857134

Level 12
Count of 0 = 11
Count of 1 = 4
Current Entropy is = 0.579915171418
Splitting on feature 21 with gain ratio 0.487751800752

Level 13
Count of 0 = 10
Count of 1 = 2
Current Entropy is = 0.450561208866
Splitting on feature 25 with gain ratio 0.38642700792

Level 14
Co


Level 5
Count of 1 = 4
Current Entropy is = 0.0
Reached leaf Node

Level 4
Count of 0 = 1
Count of 1 = 1
Current Entropy is = 0.69314718056
Splitting on feature 0 with gain ratio 0.0

Level 5
Count of 0 = 1
Current Entropy is = 0.0
Reached leaf Node

Level 5
Count of 1 = 1
Current Entropy is = 0.0
Reached leaf Node

Level 3
Count of 1 = 5
Current Entropy is = 0.0
Reached leaf Node
